# Autonomous Investment Research Agent Demo

Interactive demo notebook for the agent: Import modules from src/, run research, view reports/insights.

## Quick Setup
- Modules: src/investment_research_agent.py, src/tool_router.py, src/news_processor.py, src/analyzers.py.
- Run cells sequentially; edit symbol in Cell 3.

Oct 11, 2025: AAPL Q4 EPS $1.74 forecast, revenue $101.72B.

In [1]:
# Imports & Path Setup
import sys
import json
from datetime import datetime
import os

# Add src to path for imports
sys.path.insert(0, 'src')

# Core Agent
from investment_research_agent import InvestmentResearchAgent

print("Agent imported successfully!")

Agent imported successfully!


In [ ]:
# Demo: Run Research
symbol = 'AAPL'  # Edit symbol here (e.g., 'TSLA')
agent = InvestmentResearchAgent()

# Execute full pipeline: Plan → Tools → Chain → Route → Reflect → Learn
report = agent.research(symbol)

# Preview Report
print(f"Research for {symbol} complete!")
print(json.dumps(report, indent=2, default=str)[:800] + "\n... (truncated; see {symbol}_report.md)")3

print(f"\nLearned Insights: {report['learned_insights']}")
print(f"Plan: {report['plan']}")
print(f"Reflection: {report['reflection']['feedback']} (Avg Score: {report['reflection']['avg_score']:.2f})")

# Files Generated
print(f"\nOutputs: {symbol}_report.md (Markdown), test_output.json (JSON)")

Research for AAPL complete!
{
  "symbol": "AAPL",
  "timestamp": "2025-10-11T19:11:22.744589",
  "plan": [
    "financials",
    "prices",
    "economic",
    "news",
    "edgar",
    "economic"
  ],
  "results": {
    "financials": {
      "financials": {
        "financials": {
          "revenue": 408624988160,
          "pe_ratio": 37.16212,
          "gross_profit": 190739005440,
          "market_cap": 3639902470144
        },
        "insights": [
          "high_valuation_risk: PE above historical avg",
          "strong_revenue: TTM aligns with Q4 $101.72B forecast",
          "earnings_preview: 2025-10-30, EPS $1.74, Rev $101.72B"
        ],
        "risk_score": 1
      },
      "timestamp": "2025-10-11T19:11:22.496582"
    },
    "prices": {
      "data": {
        "Open": [
          228.23777327090536,

... (truncated; see {symbol}_report.md)

Learned Insights: ['high_risk']
Plan: ['financials', 'prices', 'economic', 'news', 'edgar', 'economic']
Reflection: Avg 1.00: Exce

In [3]:
# Display MD Report (Inline)
if os.path.exists(f"{symbol}_report.md"):
    with open(f"{symbol}_report.md", 'r') as f:
        print("=== Markdown Report ===")
        print(f.read())
else:
    print("Run Cell 3 first to generate MD.")

=== Markdown Report ===
# Research Report: AAPL

**Timestamp**: 2025-10-11T19:11:22.744589

**Plan**: financials, prices, economic, news, edgar, economic

## FINANCIALS
{
  "financials": {
    "financials": {
      "revenue": 408624988160,
      "pe_ratio": 37.16212,
      "gross_profit": 190739005440,
      "market_cap": 3639902470144
    },
    "insights": [
      ...

## PRICES
{
  "data": {
    "Open": [
      228.23777327090536,
      227.64055393917852,
      232.527799450427,
      230.52712470524975,
      232.348638945834,
      235.0858880500012,
      233.36389625282...

## ECONOMIC
{
  "data": {
    "gdp_growth": "3.3%",
    "quarter": "Q2 2025",
    "value": 30485.729
  },
  "timestamp": "2025-10-11T19:11:22.744587"
}...

## NEWS
{
  "news": {
    "raw": [
      {
        "title": "Apple (AAPL) Stock Might Be Rotting - 24/7 Wall St.",
        "summary": "Doug McIntyre and Lee Jackson both say Apple's innovation has stalled sin...

## EDGAR
{
  "filing": {
    "latest": "10

In [4]:
# Inspect Learning (Memory)
print("=== Agent Memory (Insights Across Runs) ===")
print(json.dumps(agent.memory, indent=2))

# Re-run to see bias (e.g., 'high_risk' inserts 'economic' early)
report_rerun = agent.research(symbol)
print(f"\nRe-run Plan (Biased?): {report_rerun['plan'][:3]}...")

=== Agent Memory (Insights Across Runs) ===
{
  "insights": {
    "AAPL": [
      "high_risk",
      "analyzed on 2025-10-11T18:49:56.328236",
      "earnings_preview_needed",
      "high_risk",
      "analyzed on 2025-10-11T18:50:59.730617",
      "high_risk",
      "analyzed on 2025-10-11T18:51:26.502826",
      "high_risk",
      "analyzed on 2025-10-11T19:11:22.744611"
    ],
    "TSLA": [
      "high_risk",
      "analyzed on 2025-10-11T18:51:26.873810",
      "earnings_preview_needed"
    ]
  },
  "runs": [
    {
      "timestamp": "2025-10-11T18:49:56.329587",
      "symbol": "init"
    },
    {
      "timestamp": "2025-10-11T18:50:59.730922",
      "symbol": "init"
    },
    {
      "timestamp": "2025-10-11T18:51:26.503206",
      "symbol": "init"
    },
    {
      "timestamp": "2025-10-11T19:11:22.744852",
      "symbol": "init"
    }
  ]
}

Re-run Plan (Biased?): ['financials', 'prices', 'economic']...


In [5]:
# Multi-Symbol Demo (E2E)
symbols = ['AAPL', 'TSLA']
for sym in symbols:
    report = agent.research(sym)
    print(f"\n{sym}: Score {report['reflection']['avg_score']:.2f}, Insights {report['learned_insights']}")

print("\nMemory Updated:", json.dumps({k: len(v) for k, v in agent.memory['insights'].items()}, indent=2))


AAPL: Score 1.00, Insights ['high_risk']

TSLA: Score 1.00, Insights ['high_risk']

Memory Updated: {
  "AAPL": 13,
  "TSLA": 5
}


In [6]:
# Quick Unit Tests (Post-Demo Validation)
def test_plan_bias():
    agent.memory['insights']['AAPL'] = ['high_risk']
    plan = agent.plan_research('AAPL')
    return 'economic' in plan[1]

def test_reflect_depth():
    mock_ts = datetime.now().isoformat()
    mock_results = {
        'prices': {'data': {}, 'timestamp': mock_ts},
        'financials': {'financials': {'insights': ['test']}, 'timestamp': mock_ts}
    }
    agent.plan = ['prices', 'financials']
    report = agent.generate_report(mock_results, 'AAPL')
    reflection = agent.self_reflect(report)
    return reflection['scores']['depth'] >= 0.5

print(f"Plan Bias Test: {test_plan_bias()}")
print(f"Reflection Depth Test: {test_reflect_depth()}")

# Chaining Test (News)
from news_processor import NewsProcessor
np = NewsProcessor()
mock_news = [{"title": "Test", "summary": "bad news fall"}]
processed = np.chain_news(mock_news)
print(f"News Chaining Test: {processed['classified'][0]['sentiment'] == 'negative'}")

Plan Bias Test: True
Reflection Depth Test: True
News Chaining Test: True


In [7]:
from mermaid import Mermaid
# Mermaid Diagram (Paste to Markdown Viewer)
diagram = Mermaid("""
graph TD
    A[Symbol Input] --> B[Plan: Dynamic + Memory Bias]
    B --> C[Tools: yfinance + Fallbacks]
    C --> D[Chain News: Preprocess/Classify/Extract/Summarize]
    D --> E[Route: EarningsAnalyzer Insights]
    E --> F[Reflect: Score Criteria]
    F --> G{<0.75?}
    G -->|Yes| H[Optimize: Re-Route Low Step]
    H --> E
    G -->|No| I[Learn: Flag to Memory]
    I --> J[Output: JSON/MD]
""")
diagram  # Renders SVG below

In [8]:
# Streamlit UI Code (Save to app.py & Run: streamlit run app.py)
ui_code = """
import streamlit as st
from investment_research_agent import InvestmentResearchAgent

st.title("Agent Demo UI")
symbol = st.text_input("Symbol:", "AAPL").upper()
if st.button("Research"):
    agent = InvestmentResearchAgent()
    report = agent.research(symbol)
    st.json(report)
    st.download_button("Download MD", data=open(f"{symbol}_report.md").read(), file_name=f"{symbol}.md")
"""
print(ui_code)


import streamlit as st
from investment_research_agent import InvestmentResearchAgent

st.title("Agent Demo UI")
symbol = st.text_input("Symbol:", "AAPL").upper()
if st.button("Research"):
    agent = InvestmentResearchAgent()
    report = agent.research(symbol)
    st.json(report)
    st.download_button("Download MD", data=open(f"{symbol}_report.md").read(), file_name=f"{symbol}.md")

